In [37]:
import numpy as np
import random
from keras import regularizers
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import crocoddyl
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
crocoddyl.switchToNumpyMatrix()

class regression():
    def __init__(self,
                 n_trajectories: int = 10000,
                 n_hidden: int = 5,
                 state_weight: float = 1., 
                 control_weight: float = .3, 
                 nodes: int = 3,
                 save_trajectories: bool = False,
                 save_model: bool = False,
                 plot: bool = False):
        
        self.__n_trajectories = n_trajectories
        self.__state_weight = state_weight
        self.__control_weight = control_weight
        self.__nodes = nodes
        self.__n_hidden = n_hidden
        self.save_trajectories = save_trajectories
        self.save_model = save_model
        self.plot = plot
        
        
    def _generate_trajectories(self):
        """
        This could be done better with pool. But since we are generating a maximum of 10K trajectories,
        there' no need for pool
        """

        starting_configurations = []
        optimal_trajectories = []
        feasible_trajectories = 0
        for _ in range(self.__n_trajectories):
            initial_config = np.matrix([random.uniform(-2.1, 2.),
                                        random.uniform(-2.1, 2.),
                                        random.uniform(0, 1)]).T
            model = crocoddyl.ActionModelUnicycle()
            model.costWeights = np.matrix([self.__state_weight, self.__control_weight]).T
            problem = crocoddyl.ShootingProblem(initial_config, [ model ] * self.__nodes, model)
            ddp = crocoddyl.SolverDDP(problem)
            ddp.solve()
            if ddp.isFeasible:
                state = np.squeeze(np.asarray(ddp.xs[1:]))
                control = np.squeeze(np.asarray(ddp.us))
                feasible_trajectories += 1
                complete_trajectory = np.hstack((state, control))      
                starting_configurations.append(np.squeeze(np.asarray(initial_config)))
                optimal_trajectories.append(complete_trajectory.flatten())

        starting_configurations = np.asarray(starting_configurations)
        optimal_trajectories = np.asarray(optimal_trajectories)
        if self.save_trajectories:
            f = open('../../data/x_data.pkl', 'wb')
            cPickle.dump(starting_configurations, f, protocol=cPickle.HIGHEST_PROTOCOL)
            g = open("../../data/y_data.pkl", "wb")
            cPickle.dump(optimal_trajectories, g, protocol=cPickle.HIGHEST_PROTOCOL)
            f.close(), g.close()



        return starting_configurations, optimal_trajectories, feasible_trajectories
    
    def keras_net(self, save: bool = False):
        starting_configurations, optimal_trajectories, feasible_trajectories = self._generate_trajectories()
        x_train = starting_configurations[0 : 9000, :]
        y_train = optimal_trajectories[0 : 9000, :]
        x_test = starting_configurations[9000 :, :]
        y_test = optimal_trajectories[9000 :, :]
        model = Sequential()
        model.add(Dense(256,
                        activation = 'relu',
                        input_shape = (starting_configurations.shape[1],)))
        for _ in range(self.__n_hidden):
            model.add(Dense(256,
                            activation = "tanh",
                            kernel_initializer='random_uniform',
                            kernel_regularizer=regularizers.l2(0.01),
                            activity_regularizer=regularizers.l1(0.01)))            
            model.add(Dropout(0.25))
            
        model.add(Dense(optimal_trajectories.shape[1], 
                        activation = 'linear'))
        
        sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
        model.compile(loss='mean_squared_error',
              optimizer=sgd,
              metrics=['mean_squared_error', "mean_absolute_error"])
        
        print('Train...')
        
        model.fit(x_train, 
                  y_train,
                  batch_size = 32,
                  epochs = 200,
                  verbose = 0
                  )
        
        score = model.evaluate(x_test, y_test, batch_size = 16, use_multiprocessing=True)
        
        print(score)
        return model
        

regression = regression()
neural_net = regression.keras_net()

Using TensorFlow backend.
/home/a/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/a/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/a/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/a/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9628199341609124880
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 13259392167708635828
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 5181705501328764262
physical_device_desc: "device: XLA_CPU device"
]
Train...

1000/1000 [==============================] - 0s 110us/step
[1.7514176473617553, 1.1397342681884766, 0.7316433787345886]


In [38]:
neural_net.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               1024      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)              

In [41]:
neural_net.output_shape[1]

15

In [ ]:
optimal_trajectories =  []
starting_configurations = []

initial_config = np.matrix([random.uniform(-2.1, 2.),
                            random.uniform(-2.1, 2.),
                            random.uniform(0, 1)]).T
model = crocoddyl.ActionModelUnicycle()
problem = crocoddyl.ShootingProblem(initial_config, [ model ] * 3 , model)
ddp = crocoddyl.SolverDDP(problem)
ddp.solve()
if ddp.isFeasible:
    ddp_xs = np.squeeze(np.asarray(ddp.xs))
    ddp_us = np.squeeze(np.asarray(ddp.us))
    #feasible_trajectories += 1
    x = ddp_xs[1:,0]
    y = ddp_xs[1:,1]
    theta = ddp_xs[1:,2]
    velocity = ddp_us[:,0]
    torque = ddp_us[:,1]
    optimal_trajectory = np.hstack((x, y, theta, velocity, torque))
    starting_configurations.append(np.squeeze(np.asarray(initial_config)))
    optimal_trajectories.append(optimal_trajectory)

In [ ]:
?ddp.solve

In [100]:
result = neural_net.predict(np.asarray(initial_config).T)
result = result.reshape(5, 3)
print(result)
states, controls = [], []


states.append(np.asarray(initial_config).T)
for i in range(result.shape[1] ):
    xs = result[0:3, i]
    us = result[3:5, i]
    print(xs)
    states.extend(xs)
    controls.extend(us)


states
"""
model = crocoddyl.ActionModelUnicycle()
#model.costWeights = np.matrix([self.__state_weight, self.__control_weight]).T
problem = crocoddyl.ShootingProblem(initial_config, [ model ] * 3, model)
ddp = crocoddyl.SolverDDP(problem)
ddp.solve(states, controls)
print(ddp.iter)
"""

[[-0.02329496 -0.05480011  0.38448283]
 [ 0.09460708 -1.1553668  -0.01770783]
 [-0.04531938  0.3147722   0.05512296]
 [-0.6836708  -0.01413833 -0.04145358]
 [ 0.2857213   0.02421111 -0.31900278]]
[-0.02329496  0.09460708 -0.04531938]
[-0.05480011 -1.1553668   0.3147722 ]
[ 0.38448283 -0.01770783  0.05512296]


'\nmodel = crocoddyl.ActionModelUnicycle()\n#model.costWeights = np.matrix([self.__state_weight, self.__control_weight]).T\nproblem = crocoddyl.ShootingProblem(initial_config, [ model ] * 3, model)\nddp = crocoddyl.SolverDDP(problem)\nddp.solve(states, controls)\nprint(ddp.iter)\n'

In [99]:
result = neural_net.predict(np.asarray(initial_config).T)

In [44]:
result

array([[-0.02329496, -0.05480011,  0.38448283,  0.09460708, -1.1553668 ,
        -0.01770783, -0.04531938,  0.3147722 ,  0.05512296, -0.6836708 ,
        -0.01413833, -0.04145358,  0.2857213 ,  0.02421111, -0.31900278]],
      dtype=float32)

In [45]:
result = result.reshape(5, 3)

In [52]:
result

array([[-0.02329496, -0.05480011,  0.38448283],
       [ 0.09460708, -1.1553668 , -0.01770783],
       [-0.04531938,  0.3147722 ,  0.05512296],
       [-0.6836708 , -0.01413833, -0.04145358],
       [ 0.2857213 ,  0.02421111, -0.31900278]], dtype=float32)

In [62]:
ddpx = result[0:3,0]

In [60]:
ddpu = result[3:5,0]

In [63]:
ddpx

array([-0.02329496,  0.09460708, -0.04531938], dtype=float32)

In [61]:
ddpu

array([-0.6836708,  0.2857213], dtype=float32)

In [69]:
ddp_xsss = []
ddp_xsss.append(np.asarray(initial_config).T)
for i in range(result.shape[1] ):
    ddp_xsss.append(result[0:3,i])
    

In [71]:
ddp_xsss

[array([[-1.99219852, -0.21884292,  0.43390863]]),
 array([-0.02329496,  0.09460708, -0.04531938], dtype=float32),
 array([-0.05480011, -1.1553668 ,  0.3147722 ], dtype=float32),
 array([ 0.38448283, -0.01770783,  0.05512296], dtype=float32)]

In [72]:
ddp_usss = []
for i in range(result.shape[1] ):
    ddp_usss.append(result[3:5,i])
    

In [73]:
ddp_usss

[array([-0.6836708,  0.2857213], dtype=float32),
 array([-0.01413833,  0.02421111], dtype=float32),
 array([-0.04145358, -0.31900278], dtype=float32)]

In [ ]:
result_v

In [ ]:
result_w

In [ ]:
?ddp.solve()

In [6]:
import crocoddyl, numpy as np, random

starting_configurations = []
optimal_trajectories = []
feasible_trajectories = 0


for _ in range(1):
    initial_config = np.matrix([random.uniform(-2.1, 2.),
                                random.uniform(-2.1, 2.),
                                random.uniform(0, 1)]).T
    model = crocoddyl.ActionModelUnicycle()
    #model.costWeights = np.matrix([self.__state_weight, self.__control_weight]).T
    problem = crocoddyl.ShootingProblem(initial_config, [ model ] * 10, model)
    ddp = crocoddyl.SolverDDP(problem)
    ddp.solve()
    if ddp.isFeasible:
        state = np.squeeze(np.asarray(ddp.xs[1:]))
        control = np.squeeze(np.asarray(ddp.us))
        feasible_trajectories += 1
        complete_trajectory = np.hstack((state, control))      
        starting_configurations.append(np.squeeze(np.asarray(initial_config)))
        optimal_trajectories.append(complete_trajectory)
        
        return 

In [23]:
state = np.squeeze(np.asarray(ddp.xs[1:]))

In [24]:
control = np.squeeze(np.asarray(ddp.us))

In [25]:
complete_tra = np.hstack((state, control))

In [35]:
state

array([[-1.07194027e+00,  2.07568212e-01, -9.60869705e-02],
       [-4.06170166e-01,  1.43398773e-01, -1.29187910e-01],
       [-1.31904203e-01,  1.07768488e-01, -7.73940068e-02],
       [-3.59080042e-02,  1.00324088e-01, -3.65488553e-02],
       [-7.45210159e-03,  9.92835941e-02, -1.53240771e-02],
       [-5.74350558e-04,  9.91781907e-02, -6.01285285e-03],
       [ 5.16586748e-04,  9.91716310e-02, -2.28170604e-03],
       [ 4.17741518e-04,  9.91718565e-02, -8.61671348e-04],
       [ 2.28699947e-04,  9.91720194e-02, -3.40802806e-04],
       [ 1.31249013e-04,  9.91720526e-02, -1.70401403e-04]])

In [32]:
rl = complete_tra.flatten()

In [34]:
rl.reshape(10, 5)

array([[-1.07194027e+00,  2.07568212e-01, -9.60869705e-02,
         1.01424933e+01, -5.29995602e+00],
       [-4.06170166e-01,  1.43398773e-01, -1.29187910e-01,
         6.68855397e+00, -3.31009391e-01],
       [-1.31904203e-01,  1.07768488e-01, -7.73940068e-02,
         2.76570670e+00,  5.17939028e-01],
       [-3.59080042e-02,  1.00324088e-01, -3.65488553e-02,
         9.62844190e-01,  4.08451515e-01],
       [-7.45210159e-03,  9.92835941e-02, -1.53240771e-02,
         2.84749192e-01,  2.12247782e-01],
       [-5.74350558e-04,  9.91781907e-02, -6.01285285e-03,
         6.87855865e-02,  9.31122423e-02],
       [ 5.16586748e-04,  9.91716310e-02, -2.28170604e-03,
         1.09095703e-02,  3.73114681e-02],
       [ 4.17741518e-04,  9.91718565e-02, -8.61671348e-04,
        -9.88454876e-04,  1.42003469e-02],
       [ 2.28699947e-04,  9.91720194e-02, -3.40802806e-04,
        -1.89041641e-03,  5.20868542e-03],
       [ 1.31249013e-04,  9.91720526e-02, -1.70401403e-04,
        -9.74509396e-04